In [1]:
%load_ext autoreload
%autoreload 2

from diffusers import ControlNetModel
import torch

In [2]:
from text3d2video.generative_rendering import GenerativeRenderingPipeline

sd_repo = "runwayml/stable-diffusion-v1-5"
controlnet_repo = "lllyasviel/control_v11f1p_sd15_depth"

device = torch.device("cuda")
dtype = torch.float16

controlnet = ControlNetModel.from_pretrained(controlnet_repo, torch_dtype=dtype).to(
    device
)

pipe = GenerativeRenderingPipeline.from_pretrained(
    sd_repo, controlnet=controlnet, torch_dtype=dtype
).to(device)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from text3d2video.artifacts.animation_artifact import AnimationArtifact

animation = AnimationArtifact.from_wandb_artifact_tag("joyful-jump:latest")

frame_nums = animation.frame_nums(50)
frames = animation.load_frames(frame_nums)
verts_uvs, faces_uvs = animation.texture_data()

/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [5]:
n_frames = 60
latents = pipe.prepare_latents(n_frames, 512)

latents_stacked = torch.stack([latents] * 2)
latents_stacked.shape

chunk_size = 10

for chunk_latents in latents_stacked.split(chunk_size, dim=1):
    print(chunk_latents.shape)

torch.Size([2, 10, 4, 64, 64])
torch.Size([2, 10, 4, 64, 64])
torch.Size([2, 10, 4, 64, 64])
torch.Size([2, 10, 4, 64, 64])
torch.Size([2, 10, 4, 64, 64])
torch.Size([2, 10, 4, 64, 64])


In [10]:
torch.split(torch.arange(0, 60), 10)

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39]),
 tensor([40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59]))